<a href="https://colab.research.google.com/github/suhyehye/dacon/blob/main/%EC%9E%A1%EC%BC%80%EC%96%B4%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글 드라이브
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
#전역변수

DATA_PATH = "/content/drive/MyDrive/dacon/job_care/data/"
SUBMIT_PATH = "/content/drive/MyDrive/dacon/job_care/submit/"
SEED = 42

In [3]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.2 MB/s 


In [4]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier

print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")
print(f"- sklearn: {sklearn.__version__}")

- os: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
- python: 3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]
- pandas: 1.3.5
- numpy: 1.21.5
- sklearn: 1.0.2


In [11]:
train_data = pd.read_csv(f'train.csv')
test_data = pd.read_csv(f'test.csv')

code_d = pd.read_csv(f'속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv(f'속성_H_코드.csv')
code_l = pd.read_csv(f'속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [12]:
train_data.head()

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_d_1,person_prefer_d_2,person_prefer_d_3,person_prefer_e,person_prefer_f,person_prefer_g,person_prefer_h_1,person_prefer_h_2,person_prefer_h_3,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target
0,0,True,True,True,False,False,False,1,4,3,5,275,370,369,8,1,1,4,95,59,3,3,10,2,1,2,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1
1,1,False,False,False,True,True,False,1,3,4,1,114,181,175,4,1,1,131,101,96,1,3,5,1,1,2,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0
2,2,False,False,False,True,False,False,2,0,3,5,464,175,452,3,1,1,54,263,56,3,1,10,2,1,1,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0
3,3,False,False,False,True,False,False,2,0,2,5,703,705,704,3,1,1,72,227,2,1,3,5,1,1,2,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0
4,4,True,True,True,False,False,False,1,3,4,5,275,370,369,4,1,1,214,210,209,1,1,10,2,1,2,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0


In [13]:
code_d.head()

,속성 D 코드,속성 D 세분류코드,속성 D 소분류코드,속성 D 중분류코드
0,4,4,3,2
1,5,5,3,2
2,7,7,6,2
3,8,8,6,2
4,9,8,6,2


In [14]:
code_h.head()

,속성 H 코드,속성 H 중분류코드,속성 H 대분류코드
0,2,315,1
1,4,316,3
2,5,317,3
3,6,318,3
4,7,319,3


In [17]:
code_d.columns = ['attribute_d','attribute_d_d','attribute_d_s','attribute_d_m']
code_h.columns = ['attribute_h','attribute_h_p','attribute_h_s']
code_l.columns = ['attribute_l','attribute_l_d','attribute_l_s','attribute_l_m','attribute_l_l']

In [ ]:
#속성코드 데이터 merge함수

def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:
  df = df.copy()
  df_code = df_code.copy()
  df_code.columns.all